In [2]:
!pip install lightgbm ==3.2.0

  Found existing installation: lightgbm 4.1.0
    Uninstalling lightgbm-4.1.0:
      Successfully uninstalled lightgbm-4.1.0


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgbm

In [4]:
dataset=pd.read_csv("insurance_pre.csv")

In [5]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [7]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [8]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [9]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

In [10]:
dependent=dataset[['charges']]

In [92]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
regressor=LGBMRegressor()
param_grid={'n_estimators': [50,100,1000],
            'num_leaves': [10],
            'max_depth': [3]}
grid = GridSearchCV(regressor,param_grid,n_jobs =-1,verbose=3,refit=True)

In [93]:
grid.fit(independent,dependent)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'max_depth': [3], 'n_estimators': [50, 100, 1000],
                         'num_leaves': [10]},
             verbose=3)

In [94]:
re=grid.cv_results_
print(re)

{'mean_fit_time': array([0.03437347, 0.05815744, 0.20429997]), 'std_fit_time': array([0.00625017, 0.01537935, 0.03771745]), 'mean_score_time': array([0.01562467, 0.01445642, 0.02187366]), 'std_score_time': array([7.00804637e-07, 9.27594973e-03, 7.65389636e-03]), 'param_max_depth': masked_array(data=[3, 3, 3],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[50, 100, 1000],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'param_num_leaves': masked_array(data=[10, 10, 10],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'max_depth': 3, 'n_estimators': 50, 'num_leaves': 10}, {'max_depth': 3, 'n_estimators': 100, 'num_leaves': 10}, {'max_depth': 3, 'n_estimators': 1000, 'num_leaves': 10}], 'split0_test_score': array([0.88500922, 0.88149651, 0.85463326]), 'split1_test_score': array([0.81543017, 0.81159722, 0.7802

In [95]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,param_num_leaves,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.034373,0.006250,0.015625,7.008046e-07,3,50,10,"{'max_depth': 3, 'n_estimators': 50, 'num_leav...",0.885009,0.815430,0.894452,0.852458,0.866767,0.862823,0.027791,1
1,0.058157,0.015379,0.014456,9.275950e-03,3,100,10,"{'max_depth': 3, 'n_estimators': 100, 'num_lea...",0.881497,0.811597,0.893434,0.851057,0.860707,0.859658,0.028294,2
2,0.204300,0.037717,0.021874,7.653896e-03,3,1000,10,"{'max_depth': 3, 'n_estimators': 1000, 'num_le...",0.854633,0.780276,0.862542,0.832763,0.839521,0.833947,0.028840,3


In [96]:
regressor = lgbm.LGBMRegressor(n_estimators=100)
regressor.fit(independent,dependent)

LGBMRegressor()

In [97]:
print("The R Score value for the best Parameter{}:".format(grid.best_params_))

The R Score value for the best Parameter{'max_depth': 3, 'n_estimators': 50, 'num_leaves': 10}:


In [98]:
print(independent.columns)

Index(['age', 'bmi', 'children', 'sex_male', 'smoker_yes'], dtype='object')


In [99]:
import pickle
filename="finalized_Model_lightgbm.sav"
pickle.dump(grid,open(filename,'wb'))

In [100]:
loaded_model=pickle.load(open("finalized_Model_lightgbm.sav",'rb'))
prediction = [[19, 27.900, 0,0,1]]
Future_prediction = pd.DataFrame(prediction, columns=['age', 'bmi', 'children', 'sex_male', 'smoker_yes'])
result = loaded_model.predict(Future_prediction)
print(result)

[18158.78770548]
